In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from copy import deepcopy

from sklearn.preprocessing import LabelEncoder, normalize
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from econml.grf import CausalForest
from sklearn.linear_model import LassoCV

import warnings
warnings.filterwarnings("ignore")

In [20]:
# load data
welfare_raw = pd.read_csv("welfarelabel.csv", low_memory=False)
labels = welfare_raw['y'].values
treatments = welfare_raw['w']
welfare_raw.drop(columns=["_merge", 'y', 'id'], inplace=True)
welfare_raw

,year,wrkstat,hrs1,hrs2,evwork,occ,prestige,wrkslf,wrkgovt,commute,...,preteen_miss,teens_miss,adults_miss,unrelat_miss,earnrs_miss,income_miss,rincome_miss,income86_miss,partyid_miss,polviews_miss
0,1986,working fulltime,40.000000,38.613701,1.1395408,270.00000,44.000000,someone else,private,60,...,0,0,0,0,0,0,0,0,0,0
1,1986,keeping house,41.733318,38.613701,1,195.00000,51.000000,someone else,private,10,...,0,0,0,1,0,0,1,0,0,0
2,1986,working fulltime,40.000000,38.613701,1.1395408,184.00000,51.000000,someone else,private,35,...,0,0,0,1,0,0,0,0,0,0
3,1986,retired,41.733318,38.613701,1,311.00000,36.000000,someone else,1,25,...,0,0,0,0,0,0,1,0,0,0
4,1986,working parttime,41.733318,38.613701,1.1395408,449.41599,40.335918,someone else,1.8203658,25,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36496,2010,retired,41.733318,38.613701,1,449.41599,40.335918,someone else,private,20.245865,...,0,0,0,0,0,0,1,1,0,0
36497,2010,retired,41.733318,38.613701,1,449.41599,40.335918,someone else,private,20.245865,...,0,0,0,0,0,0,1,1,0,0
36498,2010,working fulltime,40.000000,38.613701,1.1395408,449.41599,40.335918,someone else,private,20.245865,...,0,0,0,1,0,0,0,1,0,0
36499,2010,working fulltime,49.000000,38.613701,1.1395408,449.41599,40.335918,someone else,private,20.245865,...,0,0,0,1,0,0,1,1,0,0


In [29]:
welfare_raw.loc[(welfare_raw['year'] == 1994) & (welfare_raw['w'] == 1)]

,year,wrkstat,hrs1,hrs2,evwork,occ,prestige,wrkslf,wrkgovt,commute,...,preteen_miss,teens_miss,adults_miss,unrelat_miss,earnrs_miss,income_miss,rincome_miss,income86_miss,partyid_miss,polviews_miss
10802,1994,keeping house,41.733318,38.613701,1,449.41599,40.335918,someone else,1.8203658,20.245865,...,0,0,0,0,0,0,1,1,0,0
10804,1994,working fulltime,50.000000,38.613701,1.1395408,449.41599,40.335918,someone else,1.8203658,20.245865,...,0,0,0,0,0,0,0,1,0,0
10809,1994,working fulltime,45.000000,38.613701,1.1395408,449.41599,40.335918,someone else,1.8203658,20.245865,...,0,0,0,1,0,0,0,1,0,0
10810,1994,working fulltime,40.000000,38.613701,1.1395408,449.41599,40.335918,someone else,1.8203658,20.245865,...,0,0,0,0,0,0,0,1,0,0
10812,1994,working fulltime,40.000000,38.613701,1.1395408,449.41599,40.335918,someone else,1.8203658,20.245865,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13782,1994,other,41.733318,38.613701,1,449.41599,40.335918,someone else,1.8203658,20.245865,...,0,0,0,1,0,0,1,1,0,0
13783,1994,retired,41.733318,38.613701,1,449.41599,40.335918,1,1.8203658,20.245865,...,0,0,0,1,0,0,1,1,0,0
13788,1994,working fulltime,40.000000,38.613701,1.1395408,449.41599,40.335918,someone else,1.8203658,20.245865,...,0,0,0,0,0,0,0,1,0,0
13790,1994,retired,41.733318,38.613701,1,449.41599,40.335918,someone else,1.8203658,20.245865,...,0,0,0,0,0,0,1,1,0,0


In [3]:
def cleanWelfare(welfare_raw):
    welfare = welfare_raw.copy()

    toClean = set(['commute', 'childs', 'age', 'preteen', 'adults', 'unrelat', 'earnrs'])
    toEncode = set(['year', 'occ'])

    encoders = {} # want a dictionary to keep track of columns and their encoded values so can decode when done

    for column in welfare:
        if '_' not in column:
            if column in toEncode:
                le = LabelEncoder()
                le.fit(welfare[column])
                welfare[column] = le.transform(welfare[column])
                encoders[column] = le 
            elif column in toClean:
                if column == 'commute':
                    welfare[column] = pd.to_numeric(welfare[column].apply(lambda x: 97 if x == '97+ minutes' else x), errors='coerce')
                elif column == 'childs' or column == 'earnrs':
                    welfare[column] = pd.to_numeric(welfare[column].apply(lambda x: 8 if x == 'eight or more' else x), errors='coerce')
                elif column == 'age':
                    welfare[column] = pd.to_numeric(welfare[column].apply(lambda x: 89 if x == '89 or older' else x), errors='coerce')
                elif column == 'preteen' or column == 'adults' or column =='unrelat':
                    welfare[column] = pd.to_numeric(welfare[column].apply(lambda x: 8 if x == '8 or more' else x), errors='coerce')
                else:
                    continue # should never reach here
                welfare[column] = normalize(welfare[column].values.reshape(1, -1))[0] # once column converted to float, normalize
            else:
                if welfare[column].dtype == welfare['teens'].dtype: # float64 column
                    welfare[column] = normalize(welfare[column].values.reshape(1, -1))[0]
                elif welfare[column].dtype == welfare['polviews'].dtype: # object column
                    le = LabelEncoder()
                    le.fit(welfare[column])
                    welfare[column] = le.transform(welfare[column])
                    encoders[column] = le 
                else:
                    continue # should never reach here
    return welfare, encoders

In [4]:
welfare, encoders = cleanWelfare(welfare_raw)

In [5]:
welfare

,year,wrkstat,hrs1,hrs2,evwork,occ,prestige,wrkslf,wrkgovt,commute,...,preteen_miss,teens_miss,adults_miss,unrelat_miss,earnrs_miss,income_miss,rincome_miss,income86_miss,partyid_miss,polviews_miss
0,0,7,0.004845,0.005228,1,135,0.005641,2,2,0.015315,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0.005055,0.005228,0,106,0.006538,2,2,0.002552,...,0,0,0,1,0,0,1,0,0,0
2,0,7,0.004845,0.005228,1,99,0.006538,2,2,0.008934,...,0,0,0,1,0,0,0,0,0,0
3,0,3,0.005055,0.005228,0,142,0.004615,2,0,0.006381,...,0,0,0,0,0,0,1,0,0,0
4,0,8,0.005055,0.005228,1,211,0.005171,2,1,0.006381,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36496,15,3,0.005055,0.005228,0,211,0.005171,2,2,0.005168,...,0,0,0,0,0,0,1,1,0,0
36497,15,3,0.005055,0.005228,0,211,0.005171,2,2,0.005168,...,0,0,0,0,0,0,1,1,0,0
36498,15,7,0.004845,0.005228,1,211,0.005171,2,2,0.005168,...,0,0,0,1,0,0,0,1,0,0
36499,15,7,0.005935,0.005228,1,211,0.005171,2,2,0.005168,...,0,0,0,1,0,0,1,1,0,0


In [6]:
def estimate_grf(y, X, treatments, test_size=0.2, criterion='mse'):
    # split data into train and test sets 
    X_train, X_test, Y_train, Y_test, T_train, T_test = train_test_split(X, y, treatments, test_size=test_size)
        
    # specify hyperparams of model
    est = CausalForest(criterion='mse', n_estimators=1000,       
                          min_samples_leaf=1, 
                          max_depth=100, max_samples=0.5,
                          honest=True, inference=True)

    # fit model
    est.fit(X_train, T_train, Y_train)
    
    predict, sigma = est.predict_and_var(X_test)
    
    return est, predict, sigma

In [7]:
X_train, X_test, Y_train, Y_test, T_train, T_test = train_test_split(welfare, labels, treatments, test_size=0.2)
        
# specify hyperparams of model
est = CausalForest(criterion='mse', n_estimators=1000,       
                        min_samples_leaf=1, 
                        max_depth=100, max_samples=0.5,
                        honest=True, inference=True)

# fit model
est.fit(X_train, T_train, Y_train)

predict, sigma = est.predict_and_var(X_test)

est, predict, sigma

(CausalForest(max_depth=100, max_samples=0.5, min_samples_leaf=1,
              n_estimators=1000),
 array([[-0.34794342],
        [-0.39940418],
        [-0.41102332],
        ...,
        [-0.21771259],
        [-0.39153576],
        [-0.38058778]]),
 array([[[0.00120943]],
 
        [[0.00455745]],
 
        [[0.00716188]],
 
        ...,
 
        [[0.00173717]],
 
        [[0.00500777]],
 
        [[0.00829434]]]))

In [17]:
predict

array([[-0.34794342],
       [-0.39940418],
       [-0.41102332],
       ...,
       [-0.21771259],
       [-0.39153576],
       [-0.38058778]])

In [12]:
est.feature_importances()

array([6.77124612e-02, 2.59356593e-02, 1.13682537e-02, 1.25588969e-07,
       8.25922436e-04, 5.16261503e-04, 6.18278268e-04, 5.02850348e-04,
       6.30179504e-04, 4.00157686e-05, 6.24363622e-03, 3.56612378e-03,
       3.50695068e-03, 3.75480525e-04, 1.63536528e-03, 3.17623538e-04,
       2.77305091e-04, 3.90356471e-03, 1.76013660e-03, 0.00000000e+00,
       6.69925600e-05, 1.07542026e-02, 1.76370612e-03, 5.79108434e-03,
       2.47369986e-03, 8.62069507e-04, 8.14868114e-03, 1.53541012e-03,
       5.54400837e-03, 2.47052016e-03, 1.73622849e-03, 1.39597599e-03,
       1.61799844e-03, 1.80106213e-03, 8.78475161e-04, 8.48296906e-03,
       2.26973683e-04, 5.07863760e-01, 1.37394636e-03, 1.25501719e-03,
       8.18551094e-04, 1.73133793e-03, 3.82325892e-04, 1.35844972e-04,
       4.43645648e-04, 9.64355088e-04, 1.26252755e-03, 7.92911847e-04,
       2.71508445e-04, 2.29415599e-04, 3.12208323e-04, 4.70882245e-04,
       2.58490127e-04, 2.53191884e-03, 6.65136911e-03, 1.18878390e-02,
      

In [15]:
raw = pd.read_stata("GSS7214_R3.DTA", convert_categoricals=False)

In [19]:
raw['natfare'].value_counts()

3.0    15947
2.0    10260
1.0     6775
Name: natfare, dtype: int64